In [25]:
# import sys
# sys.path.append("/Users/alex/Downloads/stats/")

# import pandas as pd
# import os
# from db import connection
# from db_utils import DBUtils
# import xlrd
# from tqdm import tqdm
# import datetime
# import json

In [23]:
values = ['UN WPP - Interpolated demographic indicators by region, subregion and country, annually for 1950-2099',
 'UN WPP - Interpolated total population by broad age group, region, subregion and country, annually for 1950-2100 (thousands)',
 'UN WPP - Interpolated male population by broad age group, region, subregion and country, annually for 1950-2100 (thousands)',
 'UN WPP - Interpolated female population by broad age group, region, subregion and country, annually for 1950-2100 (thousands)',
 'UN WPP - Percentage of total population by broad age group, region, subregion and country, annually interpolated for 1950-2100',
 'UN WPP - Dependency ratios (total, child, old-age) for different age groups and for both sexes combined by region, subregion and country, annually interpolated for 1950-2100',
 'UN WPP - Total population (both sexes combined) by single age, region, subregion and country, annually for 1950-2100 (thousands)']

keys = ['Annually interpolated demographic indicators.', 'Annual total population (both sexes combined) by broad age groups. Data are presented in thousands.', 'Annual male population by broad age groups. Data are presented in thousands.', 'Annual female population by broad age groups. Data are presented in thousands.', 'Percentage of annual total population (both sexes combined) by broad age group.', 'Annual dependency ratios (total, child, old-age) for different age groups and for both sexes combined.', 'Annual population by single age - Both Sexes. De facto population as of 1 July of the year indicated classified by single age (0, 1, 2, ..., 99, 100+). Data are presented in thousands.']

datasets_dict = {}
for x,y in zip(keys, values):
    datasets_dict[x] = y
    

In [26]:
# with connection as c:
#     db = DBUtils(c)
    
#     # upsert datasets
#     dataset_name_ids = {}
#     for f in tqdm(os.listdir('sources/')):
#         if f == '.DS_Store':
#             continue
#         data = pd.read_excel('sources/'+f)
#         val = data[data.columns[0]][8]
#         index_to_remove = val.find(":")
#         res = "UN WPP - " + val[index_to_remove+2:]
#         dataset_id = db.upsert_dataset(name=res, namespace="unwpp", user_id=15)
#         dataset_name_ids[res] = dataset_id
        
#     # upsert sources
    
#     dataset_to_source_ids = {}
#     source_name = "United Nations – Population Division (2019 Revision)"
#     for addInfo, dataset_name in datasets_dict.items():
#         description = {}
#         description['dataPublishedBy'] = "United Nations, Department of Economic and Social Affairs, Population Division (2019). World Population Prospects: The 2019 Revision, DVD Edition."
#         description['dataPublisherSource'] = None
#         description['link'] = 'https://population.un.org/wpp2019/Download/Standard/Interpolated/'
#         description['retrievedDate'] = datetime.datetime.now().strftime("%d-%b-%Y")
#         description['additionalInfo'] = addInfo
        
#         source_id = db.upsert_source(name=source_name, description=json.dumps(description), dataset_id=dataset_name_ids[dataset_name])
#         dataset_to_source_ids[dataset_name_ids[dataset_name]] = source_id
    


100%|██████████| 7/7 [02:33<00:00, 20.38s/it]

In [27]:
# dataset_to_source_ids

{4219: 17021,
 4217: 17022,
 2745: 17023,
 2748: 17024,
 4221: 17025,
 4223: 17026,
 4220: 17027}

# Preparing csvs

## Datasets

In [28]:
names,ids = [], []
i = 0
for f in tqdm(os.listdir('data/')):
        if f == '.DS_Store':
            continue
        data = pd.read_excel('data/'+f)
        val = data[data.columns[0]][8]
        index_to_remove = val.find(":")
        res = "UN WPP - " + val[index_to_remove+2:]
        names.append(res)
        ids.append(i)
        i+=1
datasets = pd.DataFrame()
datasets['id'] = ids
datasets['name'] = names
datasets.to_csv('datasets.csv', index=False)


100%|██████████| 7/7 [02:30<00:00, 19.91s/it]

## Sources

In [32]:
names, desc, d_ids = [], [], []

source_name = "United Nations – Population Division (2019 Revision)"
for addInfo, dataset_name in datasets_dict.items():
    description = {}
    description['dataPublishedBy'] = "United Nations, Department of Economic and Social Affairs, Population Division (2019). World Population Prospects: The 2019 Revision, DVD Edition."
    description['dataPublisherSource'] = None
    description['link'] = 'https://population.un.org/wpp2019/Download/Standard/Interpolated/'
    description['retrievedDate'] = datetime.datetime.now().strftime("%d-%b-%Y")
    description['additionalInfo'] = addInfo
    dataset_id = datasets[datasets['name'] == dataset_name]['id'].values[0]

    names.append(source_name)
    desc.append(description)
    d_ids.append(dataset_id)
    
res = pd.DataFrame()
res['name'] = names
res['description'] = desc
res['dataset_id'] = d_ids
res.to_csv("sources.csv", index=False)

## Variables

In [71]:
class DataVariables():
    
    def __init__(self):
        
        
        self.i = 0
        self.ids = []
        self.names = []
        self.units = []
        self.dataset_ids = []
        self.doc_to_unit = {
                        "UN WPP - Interpolated total population by broad age group, region, subregion and country, annually for 1950-2100 (thousands)": "Thousands",
                        'UN WPP - Total population (both sexes combined) by single age, region, subregion and country, annually for 1950-2100 (thousands)': "Thousands",
                        'UN WPP - Percentage of total population by broad age group, region, subregion and country, annually interpolated for 1950-2100': "Percentage",
                        'UN WPP - Interpolated male population by broad age group, region, subregion and country, annually for 1950-2100 (thousands)': 'Thousands',
            'UN WPP - Interpolated female population by broad age group, region, subregion and country, annually for 1950-2100 (thousands)': 'Thousands',
             'UN WPP - Dependency ratios (total, child, old-age) for different age groups and for both sexes combined by region, subregion and country, annually interpolated for 1950-2100': 'Percentage'

            #'UN WPP - Interpolated demographic indicators by region, subregion and country, annually for 1950-2099': 
                        

                    }
        
        self.datasets = pd.read_csv('datasets.csv')
        self.df = pd.DataFrame()
        
    def get_variables(self, path, skiprows=8, prefix=None):
        data = pd.read_excel(path, skiprows=8)

        val = data[data.columns[0]][0]
        index_to_remove = val.find(":")
        res = "UN WPP - " + val[index_to_remove+2:]
        
        for item in data.loc[7, data.columns[8]: data.columns[-1]].values:
            if prefix:
                self.names.append(prefix + " - " + item)
            else:
                self.names.append(item)
            self.ids.append(self.i)
            self.i+=1
            self.units.append(self.doc_to_unit[res])
            self.dataset_ids.append(self.datasets[self.datasets['name'] == res]['id'].values[0])
            
    def get_df(self):
        
        self.df['id'] = self.ids
        self.df['name'] = self.names
        self.df['unit'] = self.units
        self.df['dataset_id'] = self.dataset_ids
        
        return self.df
        
        
        

In [72]:
dv = DataVariables()
for f in tqdm(os.listdir('data/')):
        if f == '.DS_Store':
            continue
        elif f == "unwpp.xlsx":
            dv.get_variables("./data/"+f, prefix="Total population by broad age group, both sexes combined (thousands)")
        elif f == "unwpp_total_population.xlsx":
            dv.get_variables("./data/"+f, prefix="Total population by age, both sexes combined (thousands)")
        elif f == "unwpp_percentage_of_total_population.xlsx":
            dv.get_variables("./data/"+f, prefix="Percentage of total population by broad age group, both sexes combined (per 100 total population)")
        elif f == "unwpp_male_population.xlsx":
            dv.get_variables("./data/"+f, prefix="Male population by broad age group (thousands)")
        #elif f == "unwpp_interpolated_demographic_indicators.xlsx":
        elif f == "unwpp_female_population.xlsx":
            dv.get_variables("./data/"+f, prefix="Female population by broad age group (thousands)")
        elif f == "unwpp_dependency_ratios.xlsx":
            dv.get_variables("./data/"+f, prefix="Dependency ratios (both sexes combined) for different age groups")
        

res = dv.get_df()






  0%|          | 0/8 [00:00<?, ?it/s]



 12%|█▎        | 1/8 [00:22<02:37, 22.44s/it]



 38%|███▊      | 3/8 [00:44<01:35, 19.06s/it]



 62%|██████▎   | 5/8 [01:22<00:57, 19.04s/it]



 75%|███████▌  | 6/8 [01:45<00:40, 20.05s/it]



 88%|████████▊ | 7/8 [02:07<00:20, 20.70s/it]



100%|██████████| 8/8 [02:16<00:00, 17.37s/it]

In [74]:
res.to_csv("variables.csv", index=False)